In [1]:
import json
import requests
from bs4 import BeautifulSoup

espn = "https://www.espn.com.br/futebol/"
r = requests.get("https://www.espn.com.br/futebol/resultados/_/liga/CONMEBOL.LIBERTADORES/data/20221029")
soup = BeautifulSoup(r.text)
resumes = soup.find_all("a", {"class": "button-alt sm"})

In [2]:
for script in soup.find_all("script", {"type": None, "src": None}):
    if script.text.startswith("window.espn.scoreboardData"):
        data = json.loads(script.text[30:-337])
        break